# Esta notebook se utilizara a todo lo referido a la creación de datos


In [1]:
#Imports
from models import BinPackingGame, Box

In [2]:
container = Box(4, 4)
boxes = [Box(4, 1), Box(4, 3)]
bin_packing_game = BinPackingGame(container, boxes)
print(bin_packing_game.solve())

ResolvedBinPackingGameResult(positions=[(0, 0), (1, 0)])


### Rotaciones y Orden de las Cajas
Dependiendo de cómo un algoritmo resuelve el problema de bin packing problema, tener cajas con dimensiones (ancho, alto) alternadas o incluso el orden de las cajas puede dar distintos resultados, por lo cuál a la hora de la generación tenemos que considerar que:
1. `Box(w, h)` es distinto de `Box(h,w)` a menos que `w=h`
2. La sequencia `[Box1, Box2]` es disitinta de `[Box2, Box1]` a menos que `Box1=Box2`

In [3]:
# Comparacion de cajas y rotaciones
box1 = Box(2, 3)
box2 = Box(2, 3)
assert box1 == box2, "Las cajas son iguales"
assert hash(box1) == hash(box2), "Los hashes son iguales"
box1 = Box(2, 3)
box2 = Box(5, 4)
assert box1 != box2, "Las cajas son diferentes"
assert hash(box1) != hash(box2), "Los hashes son diferentes"
box1 = Box(2, 3)
box2 = box1.rotate()
assert box1 != box2, "Las cajas son diferentes"
assert hash(box1) != hash(box2), "Los hashes son diferentes"
box1 = Box(3, 3)
box2 = box1.rotate()
assert box1 == box2, "Las cajas son iguales"
assert hash(box1) == hash(box2), "Los hashes son iguales"
print("Comparacion de cajas y rotaciones exitosa")

# Comparacion de orden de cajas
boxes1 = [Box(2, 3), Box(3, 4)]
boxes2 = [Box(2, 3), Box(3, 4)]
assert boxes1 == boxes2, "Las sequencias son iguales"
boxes1 = [Box(2, 3), Box(3, 4)] 
boxes2 = [Box(3, 4), Box(2, 3)]
assert boxes1 != boxes2, "Las sequencias son diferentes"
print("Comparacion de orden de cajas exitosa")

Comparacion de cajas y rotaciones exitosa
Comparacion de orden de cajas exitosa


### BinPackingGame Key

Tenemos que asegurarnos que un problema es igual a otro si y solo si el contenedor es el mismo y la sequencia de cajas es la misma


In [4]:
bpm1 = BinPackingGame(Box(4, 4), [Box(4, 1), Box(4, 3)])
bpm2 = BinPackingGame(Box(4, 4), [Box(4, 1), Box(4, 3)])
assert bpm1 == bpm2, "Los juegos son iguales"
assert hash(bpm1) == hash(bpm2), "Los hashes son iguales"
bpm1 = BinPackingGame(Box(4, 4), [Box(4, 1), Box(4, 3)])
bpm2 = BinPackingGame(Box(5, 5), [Box(4, 1), Box(4, 3)])
assert bpm1 != bpm2, "Los juegos son diferentes"
assert hash(bpm1) != hash(bpm2), "Los hashes son diferentes"
bpm1 = BinPackingGame(Box(4, 4), [Box(4, 1), Box(4, 3)])
bpm2 = BinPackingGame(Box(4, 4), [Box(4, 3), Box(4, 1)])
assert bpm1 != bpm2, "Los juegos son diferentes"
assert hash(bpm1) != hash(bpm2), "Los hashes son diferentes"
print("Comparacion de juegos exitosa")

Comparacion de juegos exitosa


### Data Augmentation

Ahora que definimos que 2 problemas son considerados distintos tan solo por permutar cajas o directamente rotar una caja (incluso el contenedor), dado una solución válida del problema podemos generar muchos problemas diferentes, y también válidos!

In [5]:
from itertools import permutations, product
from typing import List

from models import ResolvedBinPackingGameResult

def generate_box_rotations(boxes):
        return [[box, box.rotate()] for box in boxes]

class BinPackingGameManager:
    def __init__(self):
        self.games = set()

    def __len__(self):
        return len(self.games)
    
    def __getitem__(self, index):
        return self.games[index]
    
    def __iter__(self):
        return iter(self.games)
    
    def add_game(self, game, max_permutations = 1000):
        """
        Agrega un juego a la lista de juegos si no existe un juego igual y agrega todas las permutaciones de las cajas
        con sus posibles rotaciones y rotando también el contenedor.
        Args:
            game (BinPackingGame): Juego a agregar
            max_permutations (int): Máximo de permutaciones a generar para el juego dado
        Returns:
            Int: 0 si el juego ya existe, N donde N es la cantidad de juegos agregados
        """
        if game in self.games:
            return 0
        rotated_containers = [game.container, game.container.rotate()]
        all_box_permutations = list(permutations(game.boxes))
        permutated_games_generated = 0
        for container in rotated_containers:
            for box_permutation in all_box_permutations:
                box_rotations = generate_box_rotations(box_permutation)
                # Generar todas las combinaciones de rotaciones para esta permutación
                for rotated_boxes in product(*box_rotations):
                    permutated_game = BinPackingGame(container, list(rotated_boxes))
                    if permutated_game not in self.games:
                        self.games.add(BinPackingGame(container, list(rotated_boxes)))
                        permutated_games_generated += 1
                        if permutated_games_generated >= max_permutations:
                            return permutated_games_generated
        return permutated_games_generated

container = Box(10, 15)
boxes = [Box(10, 4), Box(3, 6), Box(7, 2), Box(5, 5)]
game = BinPackingGame(container, boxes)
assert isinstance(game.solve(), ResolvedBinPackingGameResult), "El juego original es válido"

bpgManager = BinPackingGameManager()
games_generated = bpgManager.add_game(game, max_permutations= 1000)
print(f"{games_generated} juegos creados a partir de un juego válido con un máximo de 1000 permutaciones")
for generated_game in bpgManager:
    assert isinstance(generated_game.solve(), ResolvedBinPackingGameResult), "El juego generado es válido"
print("Todos los juegos generados son válidos")
bpgManager = BinPackingGameManager()
games_generated = bpgManager.add_game(game, max_permutations= 5)
print(f"{games_generated} juegos creados a partir de un juego válido con un máximo de 5 permutaciones")
for generated_game in bpgManager:
    print(generated_game)

384 juegos creados a partir de un juego válido con un máximo de 1000 permutaciones
Todos los juegos generados son válidos
5 juegos creados a partir de un juego válido con un máximo de 5 permutaciones
BinPackingGame(container=Box(width=10, height=15), boxes=[Box(width=10, height=4), Box(width=3, height=6), Box(width=2, height=7), Box(width=5, height=5)])
BinPackingGame(container=Box(width=10, height=15), boxes=[Box(width=10, height=4), Box(width=3, height=6), Box(width=7, height=2), Box(width=5, height=5)])
BinPackingGame(container=Box(width=10, height=15), boxes=[Box(width=10, height=4), Box(width=6, height=3), Box(width=2, height=7), Box(width=5, height=5)])
BinPackingGame(container=Box(width=10, height=15), boxes=[Box(width=4, height=10), Box(width=3, height=6), Box(width=7, height=2), Box(width=5, height=5)])
BinPackingGame(container=Box(width=10, height=15), boxes=[Box(width=10, height=4), Box(width=6, height=3), Box(width=7, height=2), Box(width=5, height=5)])


### Generación de problemas válidos

Vamos a generar un gran cantidad de problemas a partir de un sub-conjunto de problemas válidos y los guardaremos en archivo

In [6]:
from models import Box
from random_generator import RandomGenerator
from utils import save

bpgManager = BinPackingGameManager()
for width in range(6, 15):
    for height in range(6, 15):
        print(f"Generando juegos para contenedor de {width}x{height}")
        container = Box(width, height)
        random_generator = RandomGenerator(
            container,
            min_container_coverage = 0.8,
            max_container_coverage = 1,
            min_boxes_per_container = 4,
            max_boxes_per_container = 10
        )
        valid_bin_packing_games = random_generator.generate(games_to_generate= 1000)
        for game in valid_bin_packing_games:
            bpgManager.add_game(game, max_permutations = 100)
        print(f"Total de juegos generados terminado el contenedor de {width}x{height}: {len(bpgManager)}")

Generando juegos para contenedor de 6x6
Generated 1000 valid bin packing games in 2938 attempts
Total de juegos generados terminado el contenedor de 6x6: 94260
Generando juegos para contenedor de 6x7
Generated 1000 valid bin packing games in 2566 attempts
Total de juegos generados terminado el contenedor de 6x7: 190960
Generando juegos para contenedor de 6x8
Generated 1000 valid bin packing games in 2836 attempts
Total de juegos generados terminado el contenedor de 6x8: 288860
Generando juegos para contenedor de 6x9
Generated 1000 valid bin packing games in 3018 attempts
Total de juegos generados terminado el contenedor de 6x9: 387860
Generando juegos para contenedor de 6x10
Generated 1000 valid bin packing games in 3081 attempts
Total de juegos generados terminado el contenedor de 6x10: 486660
Generando juegos para contenedor de 6x11
Generated 1000 valid bin packing games in 3472 attempts
Total de juegos generados terminado el contenedor de 6x11: 586160
Generando juegos para contenedo

In [7]:
save(bpgManager.games, f'permutated_games.txt')